In [3]:
from __future__ import print_function
from scipy.special import gammaln
from scipy.special import psi
from scipy.misc import factorial
from scipy.optimize import fmin_l_bfgs_b as optim
import sys

import pandas as pd
import numpy as np
import os
import xlrd
from copy import deepcopy

In [ ]:
# mutation process
def pre_mu():

    # CNV
    pan_cnv = pd.read_csv('DrugNormal/Input/CellLine_CNV_BEMs/PANCAN_CNA_BEM.rdata.txt', header=0, index_col=0, sep='\t')
    pan_cnv.index = [str(ele) for ele in pan_cnv.index]
    pan_idx = [ele for ele in pan_cnv.columns if '(' in ele]
    npan_cnv = pan_cnv.loc[:, pan_idx]
    npan_idx = [ele[ele.index('(')+1:ele.index(')')] for ele in pan_idx]
    npan_cnv.columns = npan_idx

    # mut
    pan_sm = pd.read_csv('DrugNormal/Input/CellLines_CG_BEMs/PANCAN_SEQ_BEM.txt', header=0, index_col=0, sep='\t').T

    for ele in npan_idx:
        if ',' in ele:
            nele_l = ele.split(',')
            if len(nele_l) > 1:
                if npan_cnv[[ele]].shape[1]>1:
                    da = npan_cnv[ele].iloc[:,[0]]+npan_cnv[ele].iloc[:,[1]]
                    da[da>1] = 1
                    npan_csv = npan_cnv.drop([ele], axis=1)
                    npan_cnv[ele] = da
                    for nele in nele_l:
                        npan_cnv[nele] = npan_cnv[[ele]].iloc[:,0]
                else:
                    for nele in nele_l:
                        npan_cnv[nele] = npan_cnv[[ele]].iloc[:,0]
                    npan_cnv = npan_cnv.drop([ele], axis=1)

    pan_mu =pd.concat([pan_sm, npan_cnv.loc[pan_sm.index,:]], axis=1)
    for ele in pan_mu.columns:
        if list(pan_mu.columns).count(ele)>1:
            da = pan_mu[ele].iloc[:,[0]]
            for i in range(1,list(pan_mu.columns).count(ele)):
                da = da + pan_mu[ele].iloc[:,[i]]
            da[da>1] = 1
            pan_mu = pan_mu.drop([ele], axis=1)
            pan_mu[ele] = da
            
    #gene_l = ['AKT1','AR', 'ATM','BRAF', 'RAF1','PDGFRA', 'KIT', 'KDR','CDK4',
          'EGFR', 'ERBB2','FGFR1', 'FGFR3','MDM4', 'TP53','TOP2A','PPM1D','PIK3CA','MTOR']
    GDSCmut = pd.read_csv('DrugNormal/Input/GDSC mutation.csv', index_col=0, header=0).T
    #cai = [ele for ele in gene_l if ele not in pan_mu.columns]
    pan_mu[cai] = GDSCmut[cai]
    pan_mu = pan_mu.loc[GDSCmut.index, gene_l]
    print(pan_mu)
    pan_mu.to_csv('DrugNormal/Input/GDSC sm_cnv.csv', index=True, header=True)

In [ ]:
def pre_cnv():

In [ ]:
    pan_cnv = pd.read_excel('DrugNormal/Input/Gene_level_CN.xlsx', sheet_name='Gene_level_CN')

In [4]:
    npan_cnv = pan_cnv.iloc[1:, 4:]
    npan_cnv.index = pan_cnv.iloc[:,0][1:]
    npan_cnv.columns = pan_cnv.iloc[0,:][4:]

    x = lambda a:a[0:1]
    for col in npan_cnv:
        npan_cnv[col] = npan_cnv[col].map(x)

    npan_cnv[npan_cnv=='-'] = 100
    x = lambda a:int(a)
    for col in npan_cnv:
            npan_cnv[col] = npan_cnv[col].map(x)

    nnpan_cnv = deepcopy(npan_cnv.T)
    nnpan_cnv[nnpan_cnv==100] = np.nan
    nnpan_cnv[nnpan_cnv==0] = 5
    nnpan_cnv[nnpan_cnv<=4] = 0
    nnpan_cnv[nnpan_cnv>4] = 1


    GDSCmut = pd.read_csv('DrugNormal/Input/GDSC mutation.csv', header=0, index_col=0).T
    GDSCmut.index = [int(ele) for ele in GDSCmut.index]
    idx = nnpan_cnv.index & GDSCmut.index
    col = nnpan_cnv.columns & GDSCmut.columns
    nnpan_cnv = nnpan_cnv.loc[idx, col]
    nGDSCmut = GDSCmut.loc[idx, col]
    pan_mu = nGDSCmut + nnpan_cnv 
    pan_mu[pan_mu>1] = 1
    pan_mu.to_csv('EMva/Input/S_Ac.csv', index=True, header=True)

KeyboardInterrupt: 

In [5]:
pan_cnv.shape

NameError: name 'pan_cnv' is not defined

In [ ]:
# expression process 
def dircrete_cellline():

    S_D = pd.read_csv('DrugNormal/Input/PanCancer13tts.DEGmatrix.4TCI.csv', header=0, index_col=0)
    TCGAexp = pd.read_csv('DrugNormal/Input/TCGA expression.csv', header=0, index_col=0)
    GDSCexp = pd.read_csv('DrugNormal/Input/GDSC expression.csv', header=0, index_col=0)

    tg_sub = pd.read_csv('Pre/TCGA tissue type annotation.csv', header=0, index_col=0)
    gd_sub = pd.read_csv("DrugNormal/Input/GDSC tissue type annotation_rna.csv", header=0,index_col=0)
    sc = pd.read_csv('DrugNormal/Input/Subtype Code.csv')
    sc.columns.values[0] = 'TCGA Label'
    gd_subn = gd_sub.merge(sc)
    gd_subn.to_csv('DrugNormal/Input/GDSC tissue type annotation_ac.csv', header=True, index=False)

    nTCGAexp_idx = TCGAexp.index & S_D.index
    nTCGAexp_col = TCGAexp.columns & GDSCexp.columns & S_D.columns 
    
    nS_D = S_D.loc[nTCGAexp_idx, nTCGAexp_col]
    nTCGAexp = TCGAexp.loc[nTCGAexp_idx, nTCGAexp_col]
    nGDSCexp = GDSCexp[nTCGAexp_col]
    
    tg_subn = tg_sub.loc[tg_sub.index & nTCGAexp.index, :].iloc[:, [1]]
    tcga_sam_l = []
    coad_l = []
    for i in list(np.unique(tg_subn.iloc[:, 0])):
        print(len(tcga_sam_l), i)
        if i in [9,17]:
            coad_l.extend(tg_subn.loc[tg_subn.iloc[:, 0] == i, :].index)
        if i in [14]:
            print(i)
        if i not in [9, 14, 17]:
            tcga_sam_l.append(tg_subn.loc[tg_subn.iloc[:, 0] == i, :].index)
    tcga_sam_l.append(coad_l)

    gd_subn = pd.read_csv('DrugNormal/Input/GDSC tissue type annotation_ac.csv', index_col=0, header=0)
    gd_subnn = gd_subn.loc[gd_subn.index & nGDSCexp.index, :].iloc[:, [-1]]
    gdsc_sam_l = []
    for i in list(np.unique(gd_subnn.iloc[:, 0])):
        gdsc_sam_l.append(gd_subnn.loc[gd_subnn.iloc[:, 0] == i, :].index)

    method='max'
    nnGDSCexp = pd.DataFrame(np.zeros((0, nGDSCexp.shape[1])), columns=nGDSCexp.columns)
    nnTCGAexp = pd.DataFrame(np.zeros((0, nTCGAexp.shape[1])), columns=nTCGAexp.columns)
    
    for i in range(len(tcga_sam_l)):
        
        nTCGAexpi = nTCGAexp.loc[tcga_sam_l[i], :]
        nS_Di = nS_D.loc[tcga_sam_l[i], :]
        nGDSCexpi = nGDSCexp.loc[gdsc_sam_l[i], :]

        thres0 = nTCGAexpi[nS_Di==0]
        ptp = np.ptp(nGDSCexpi.values, axis=0) / np.ptp(nTCGAexpi.values,axis=0)

        if method == 'max':

            norm_mean = np.mean(thres0)
            col_max = np.max(thres0) 
            col_min = np.min(thres0) 
            print(col_max['A1CF'])
            nnTCGAexpi_max = nTCGAexpi - col_max[None, :]
            nnTCGAexpi_max = nnTCGAexpi_max > 0
            nnTCGAexpi_min = nTCGAexpi - col_min[None, :]
            nnTCGAexpi_min = nnTCGAexpi_min < 0
            nnTCGAexpi = nnTCGAexpi_max + nnTCGAexpi_min
            nnTCGAexp = pd.concat([nnTCGAexp, nnTCGAexpi], axis=0)
            
            nnGDSCexpi_max = nGDSCexpi - ptp * col_max[None, :]
            nnGDSCexpi_max = nnGDSCexpi_max > 0
            nnGDSCexpi_min = nGDSCexpi - ptp * col_min[None, :]
            nnGDSCexpi_min = nnGDSCexpi_min < 0
            nnGDSCexpi = nnGDSCexpi_max + nnGDSCexpi_min
            nnGDSCexp = pd.concat([nnGDSCexp, nnGDSCexpi], axis=0)
            
        if method == 'gus':
            
            norm_mean = np.mean(thres0)
            std = np.std(thres0) 

            nnTCGAexpi_max = nTCGAexpi - (norm_mean[None, :] + 1.96*std[None, :])
            nnTCGAexpi_max = nnTCGAexpi_max > 0
            nnTCGAexpi_min = nTCGAexpi - (norm_mean[None, :] - 1.96*std[None, :])
            nnTCGAexpi_min = nnTCGAexpi_min < 0
            nnTCGAexpi = nnTCGAexpi_max + nnTCGAexpi_min
            nnTCGAexp = pd.concat([nnTCGAexp, nnTCGAexpi], axis=0)
            
            nnGDSCexpi_max = nGDSCexpi - ptp*(norm_mean[None, :] + 1.96*std[None, :])
            nnGDSCexpi_max = nnGDSCexpi_max > 0
            nnGDSCexpi_min = nGDSCexpi - ptp*(norm_mean[None, :] - 1.96*std[None, :])
            nnGDSCexpi_min = nnGDSCexpi_min < 0
            nnGDSCexpi = nnGDSCexpi_max + nnGDSCexpi_min
            nnGDSCexp = pd.concat([nnGDSCexp, nnGDSCexpi], axis=0)

    nnGDSCexp.to_csv('DrugNormal/Output/nGDSCexp.csv', header=True, index=True)

In [ ]:
def plot():
    import matplotlib.pyplot as plt
    n, bins, patches = plt.hist(x=nnGDSCexp.iloc[:,2], bins='auto', color='#0504aa',alpha=0.7, rwidth=0.85, density=True)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Expression-Frequency')
    maxfreq = n.max()

In [ ]:
# drug process

def split_target():

    GDSCdrug = pd.read_excel("DrugNormal/Input/Screened_Compounds.xlsx")
    Y_A = GDSCdrug.iloc[:, [0, 3]]
    drop_l = []
    for i in range(Y_A.shape[0]):
        A_l = Y_A.iloc[i, 1].replace(' ', '').split(',')
        if len(A_l) > 1:
            drop_l.append(i)
            for j in range(len(A_l)):
                b = A_l[j] + str(i)
                Y_A.loc[b] = [Y_A.iloc[i, 0], A_l[j]]
    Y_A = Y_A.drop(Y_A.index[drop_l])
    Y_A.to_csv('DrugNormal/Output/Y_A.csv', header=True, index=False)

def drug_sga():
    A_D = pd.read_csv('DrugNormal/Input/A_D.csv', header=0, index_col=0)
    GDSCmut = pd.read_csv('DrugNormal/Input/GDSC mutation.csv',
                          header=0,
                          index_col=0)
    GDSCRe = pd.read_csv('DrugNormal/Input/GDSC drug response.csv',
                         header=0,
                         index_col=0)
    Y_A = pd.read_csv('DrugNormal/Output/Y_A.csv', header=0, index_col=None)

    nY_A_idx = [
        Y_A.iloc[:, 1].tolist().index(idx) for idx in A_D.index
        if idx in Y_A.iloc[:, 1].tolist()
    ]

    nY_A = Y_A.loc[nY_A_idx, :]

    nGDSCRe_col = GDSCRe.columns & nY_A.iloc[:, 0].tolist()
    nGDSCRe = GDSCRe.loc[:, nGDSCRe_col]

    Y_l = nY_A.iloc[:, 0].tolist()
    Y_l = [str(ele) for ele in Y_l]
    A_l = nY_A.iloc[:, 1].tolist()
    del_l = []

    for i in range(nGDSCRe.shape[1]):
        print(i)
        a = Y_l.index(nGDSCRe.columns[i])
        print(a)
        over = [A_l[a]] & nGDSCRe.columns
        if len(over) > 0:
            b = list(nGDSCRe.columns).index(A_l[a])
            nGDSCRe.iloc[:, b] = nGDSCRe.iloc[:, i] + nGDSCRe.iloc[:, b]
            del_l.append(i)
        else:
            nGDSCRe.columns.values[i] = A_l[a]

    nGDSCRe = nGDSCRe.drop(nGDSCRe.columns[del_l])

    ttcn = pd.read_csv('DrugNormal/Input/ttcn.csv', header=0, index_col=0)
    nGDSCRe.to_csv('DrugNormal/Output/nGDSCRe.csv', header=True, index=True)
    nGDSCRet_idx = nGDSCRe.index & ttcn.iloc[:, 0].tolist()
    nGDSCRe.loc[nGDSCRet_idx, :].to_csv('DrugNormal/Output/nGDSCRen.csv',
                                        header=True,
                                        index=True)

    nGDSCmut_idx = [idx for idx in GDSCmut.index if idx in nGDSCRe.columns]
    nGDSCmut = GDSCmut.loc[nGDSCmut_idx, :].T
    nGDSCmut.to_csv('DrugNormal/Output/nGDSCmut.csv', header=True, index=True)
    nGDSCmutt_idx = nGDSCmut.index & ttcn.iloc[:, 0].tolist()
    nGDSCmut.loc[nGDSCmutt_idx, :].to_csv('DrugNormal/Output/nGDSCmutn.csv',
                                          header=True,
                                          index=True)

In [ ]:
# target_process

def gene_l():
    GDSCdrug = pd.read_excel("DrugNormal/Input/Screened_Compounds.xlsx")
    GDSCRe = pd.read_csv('DrugNormal/Input/GDSC drug response.csv',
                         header=0,
                         index_col=0)
    gene_l = [
        'AKT1', 'AR', 'ATM', 'BRAF', 'RAF1', 'PDGFRA', 'KIT', 'KDR', 'CDK4',
        'ERBB2', 'EGFR', 'FGFR1', 'FGFR3', 'MDM4', 'TP53', 'TOP2A', 'PPM1D',
        'PIK3CA', 'MTOR'
    ]

    for gene in gene_l:
        Y_l = []
        col_l = []
        for i in range(GDSCdrug.shape[0]):
            A_l = GDSCdrug.iloc[i, 3]
            if gene in A_l:
                print(gene + '_' + A_l)
                Y_l.append(str(GDSCdrug.iloc[i, 0]))
                col_l.append(
                    str(GDSCdrug.iloc[i, 0]) + '-' + GDSCdrug.iloc[i, 1] +
                    ':' + GDSCdrug.iloc[i, 3])
        df = GDSCRe.loc[:, Y_l]
        df.columns = col_l
        df.to_csv('DrugNormal/Output/gene/' + gene + '.csv',
                  header=True,
                  index=True)

    ttcn = pd.read_csv('DrugNormal/Input/ttcn.csv', header=0, index_col=0)
    GDSCmut = pd.read_csv('DrugNormal/Input/GDSC mutation.csv',
                          header=0,
                          index_col=0)
    nGDSCmut = GDSCmut.loc[gene_l, :].T
    nGDSCmut.to_csv('DrugNormal/Output/nGDSCmut.csv', header=True, index=True)
    nGDSCmutt_idx = nGDSCmut.index & ttcn.iloc[:, 0].tolist()
    nGDSCmut.loc[nGDSCmutt_idx, :].to_csv('DrugNormal/Output/nGDSCmutn.csv',
                                          header=True,
                                          index=True)

def chan_ens():

    nGDSCexp = pd.read_csv('DrugNormal/Input/GDSC_expression_discrete.csv',
                           index_col=0,
                           header=0)
    A_D = pd.read_csv('DrugNormal/Input/A_D.csv', header=0, index_col=0)

    gene_en = [ele for ele in nGDSCexp.columns if len(ele) > 12]

    import mygene
    mg = mygene.MyGeneInfo()
    gene_info = mg.querymany(gene_en,
                             scopes='ensembl.gene',
                             fields='symbol',
                             species='human')

    gene_name = [ele['symbol'] for ele in gene_info if len(ele) > 2]
    gene_disc = [ele['query'] for ele in gene_info if len(ele) <= 2]

    ngene_en = [ele for ele in gene_en if ele not in gene_disc]
    col_l = list(nGDSCexp.columns)
    ngene_idx = [col_l.index(ele) for ele in ngene_en]
    for i in range(len(ngene_en)):
        col_l[col_l.index(ngene_en[i])] = gene_name[i]

    sele_l = [ele for ele in col_l if ele not in gene_disc]
    nGDSCexp.columns = col_l

    nnGDSCexp = nGDSCexp[list(np.unique(sele_l))]

    nnGDSCexp = nnGDSCexp[nnGDSCexp.columns & A_D.columns]
    ttcn = pd.read_csv('DrugNormal/Input/ttcn.csv', header=0, index_col=0)
    nnGDSCexp.to_csv('DrugNormal/Output/nGDSCexp.csv', header=True, index=True)
    nnGDSCexpt_idx = nnGDSCexp.index & ttcn.iloc[:, 0].tolist()
    nnGDSCexp.loc[nnGDSCexpt_idx, :].to_csv('DrugNormal/Output/nGDSCRexpn.csv',
                                            header=True,
                                            index=True)